<a href="https://colab.research.google.com/github/norhayatikz/data-science-testing-28Apr/blob/main/pandas_continued_hsbc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas for Exploratory Data Analysis II 

Pandas is a very useful Python library for data manipulation and exploration. We have so much more to see!

In this lesson, we'll continue exploring Pandas for EDA. Specifically: 

- Identify and handle missing values with Pandas.
- Implement groupby statements for specific segmented analysis.
- Use apply functions to clean data with Pandas.
- Concatenating objects with `.append()` and `.concat()`
- Combining objects with `.join()` and `.merge()`
- Combining timeseries objects with `.merge_ordered()`


We'll implicitly review many functions from our first Pandas lesson along the way!

## Remember the AdventureWorks Cycles Dataset?
<img align="right" src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">

Here's the Production.Product table [data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html), which is a description of the fields (columns) in the table (the .csv file we will import below):<br>
- **ProductID** - Primary key for Product records.
- **Name** - Name of the product.
- **ProductNumber** - Unique product identification number.
- **MakeFlag** - 0 = Product is purchased, 1 = Product is manufactured in-house.
- **FinishedGoodsFlag** - 0 = Product is not a salable item. 1 = Product is salable.
- **Color** - Product color.
- **SafetyStockLevel** - Minimum inventory quantity.
- **ReorderPoint** - Inventory level that triggers a purchase order or work order.
- **StandardCost** - Standard cost of the product.
- **ListPrice** - Selling price.
- **Size** - Product size.
- **SizeUnitMeasureCode** - Unit of measure for the Size column.
- **WeightUnitMeasureCode** - Unit of measure for the Weight column.
- **DaysToManufacture** - Number of days required to manufacture the product.
- **ProductLine** - R = Road, M = Mountain, T = Touring, S = Standard
- **Class** - H = High, M = Medium, L = Low
- **Style** - W = Womens, M = Mens, U = Universal
- **ProductSubcategoryID** - Product is a member of this product subcategory. Foreign key to ProductSubCategory.ProductSubCategoryID.
- **ProductModelID** - Product is a member of this product model. Foreign key to ProductModel.ProductModelID.
- **SellStartDate** - Date the product was available for sale.
- **SellEndDate** - Date the product was no longer available for sale.
- **DiscontinuedDate** - Date the product was discontinued.
- **rowguid** - ROWGUIDCOL number uniquely identifying the record. Used to support a merge replication sample.
- **ModifiedDate** - Date and time the record was last updated.


### Import Pandas

In [ ]:
import pandas as pd
import numpy as np # used for linear algebra and random sampling amongst other things
import seaborn as sns # plotting library
import matplotlib.pyplot as plt # plotting library
%matplotlib inline

### Read in the dataset

We are using the `read_csv()` method (and the `\t` separator to specify tab-delimited columns).

In [ ]:
# Read in the dataset


In [ ]:
# Output the first 3 rows


In [ ]:
# Use the appropriate Pandas attribute to output the number of rows x cols


### Reset our index (like last time)

Let's bring our `ProductID` column into the index since it's the PK (primary key) of our table and that's where PKs belong as a best practice.

In [ ]:
# Replace the auto-generated index with the ProductID column


## Handling missing data

Recall missing data is a systemic, challenging problem for data analysts and data scientists. Imagine conducting a poll, but some of the data gets lost, or you run out of budget and can't complete it! 😮<br><br>

"Handling missing data" itself is a broad topic. We'll focus on two components:

- Using Pandas to identify we have missing data
- Strategies to fill in missing data (known in the business as `imputation` or `imputing`)
- The act of filling in missing data with Pandas


### Identifying missing data

Before *handling*, we must identify we're missing data at all!

We have a few ways to explore missing data, and they are reminiscent of our Boolean filters.

In [ ]:
# True when data isn't missing


In [ ]:
# True when data is missing


Now, we may want to see null values in aggregate. We can use `sum()` to sum down a given column

In [ ]:
# Here's a nice method chaining approach to get counts of missing values


Look! We've found missing values!

How could this missing data be problematic for our analysis?

### Understanding missing data

Finding missing data is the easy part! Determining way to do next is more complicated.

Typically, we are most interested in knowing **why** we are missing data. Once we know what 'type of missingness' we have (the source of missing data), we can proceed effectively.

Let's first quantify how much data we are missing. Here is another implementation of `prod.isnull().sum()`, only wrapped with a `DataFrame` and some labels to make it a little more user-friendly:

In [ ]:
# Or we can make things pretty as follows


### Filling in missing data

How we fill in data depends largely on why it is missing (types of missingness) and what sampling we have available to us.

We may:

- Delete missing data altogether
- Fill in missing data with:
    - The mean of the column
    - The median of the column
    - A predicted amount based on other factors
- Collect more data:
    - Resample the population
    - Follow up with the authority providing data that is missing


In our case, let's focus on handling missing values in `Color`. Let's get a count of the unique values in that column. We will need to use the `dropna=False` kwarg, otherwise the `pd.Series.value_counts()` method will not count `NaN` (null) values.

In [ ]:
# Let's get a value count with the nulls included


Ahoy! We have 248 nulls!

Option 1: Drop the missing values.

In [ ]:
# Drops rows where any row has a missing value
# This does not happen *in place*, so we are not actually dropping UNLESS overriding
# the default of the inplace parameter for `dropna()`


**Important!** `pd.DataFrame.dropna()` and `pd.Series.dropna()` are very versatile! Let's look at the docs (Series is similar):

```python
Signature: pd.DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
Docstring:
Remove missing values.

See the :ref:`User Guide <missing_data>` for more on which values are
considered missing, and how to work with missing data.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Determine if rows or columns which contain missing values are
    removed.

    * 0, or 'index' : Drop rows which contain missing values.
    * 1, or 'columns' : Drop columns which contain missing value.

    .. deprecated:: 0.23.0: Pass tuple or list to drop on multiple
    axes.
how : {'any', 'all'}, default 'any'
    Determine if row or column is removed from DataFrame, when we have
    at least one NA or all NA.

    * 'any' : If any NA values are present, drop that row or column.
    * 'all' : If all values are NA, drop that row or column.
thresh : int, optional
    Require that many non-NA values.
subset : array-like, optional
    Labels along other axis to consider, e.g. if you are dropping rows
    these would be a list of columns to include.
inplace : bool, default False
    If True, do operation inplace and return None.
```

**how**: This tells us if we want to remove a row if _any_ of the columns have a null, or _all_ of the columns have a null.<br>
**subset**: We can input an array here, like `['Color', 'Size', 'Weight']`, and it will only consider nulls in those columns. This is very useful!<br>
**inplace**: This is if you want to mutate (change) the source dataframe. Default is `False`, so it will return a _copy_ of the source dataframe.

To accomplish the same thing, but implement it on our entire dataframe, we can do the following:

In [ ]:
# Drops all nulls from the Color column, but returns the entire dataframe 
# instead of just the Color column



Option 2: Fill in missing values

Traditionally, we fill missing data with a median, average, or mode (most frequently occurring). For `Color`, let's replace the nulls with the string value `NoColor`.

Let's first look at the way we'd do it with a single column, using the `pd.Series.fillna()` method:

Now let's see how we'd do it to the whole dataframe, using the `pd.DataFrame.fillna()` method. Notice the similar API between the methods with the `value` kwarg. Good congruent design, pandas development team! The full dataframe is returned, and not just a column.

But wait! There's more! We can reference any other data or formulas we want with the imputation (the value we fill the nulls with). This is very handy if you want to impute with the average or median of that column... or even another column altogether! 

Here is an example where we will the nulls of `Color` with the average value from the `ListPrice` column. This has no practical value in this application, but immense value in other applications.

They're gone! Important points:

- Don't forget to use the `inplace=True` kwarg to mutate the source dataframe (i.e. `save changes`). 
- It is helpful to not use `inplace=True` initially to ensure your code/logic is correct, prior to making permanent changes.

## Groupby Statements

In Pandas, groupby statements are similar to pivot tables in that they allow us to segment our population to a specific subset.

For example, if we want to know the average number of bottles sold and pack sizes per city, a groupby statement would make this task much more straightforward.


To think how a groupby statement works, think about it like this:

- **Split:** Separate our DataFrame by a specific attribute, for example, group by `Color`
- **Combine:** Put our DataFrame back together and return some _aggregated_ metric, such as the `sum`, `count`, or `max`.

![](http://i.imgur.com/yjNkiwL.png)

Let's try it out!

Let's group by `Color`, and get a count of products for each color.

In [ ]:
# Group by Color, giving the number of products of each color


What do we notice about this output? Are all columns the same? Why or why not?

We can see that the `.count()` method excludes nulls, and there is no way to change this with the current implementation:
```python
Signature: .count()
Docstring: Compute count of group, excluding missing values
```

As a best practice, you should either:
- fill in nulls prior to your .count(), or
- use the PK (primary key) of the table, which is guaranteed non-null

In [ ]:
# Here we can use 'x' as a dummy placeholder for nulls, simply 
# to get consistent counts for all columns


Let's find out the most expensive price for an item, by `Color`:

We can also do multi-level groupbys. This is referred to as a `Multiindex` dataframe. Here, we can see the following fields in a nested group by, with a count of Name (with nulls filled!); effectively giving us a count of the number of products for every unique Class/Style combination:

- Class - H = High, M = Medium, L = Low
- Style - W = Womens, M = Mens, U = Universal

We can also use the `.agg()` method with multiple arguments, to simulate a `.describe()` method like we used before:

## Apply functions for column operations

Apply functions allow us to perform a complex operation across an entire columns highly efficiently.

For example, let's say we want to change our colors from a word, to just a single letter. How would we do that?

The first step is writing a function, with the argument being the value we would receive from each cell in the column. This function will mutate the input, and return the result. This result will then be _applied_ to the source dataframe (if desired).

In [ ]:
# Output a list of all unique colors


In [ ]:
# Create a function to convert color names to single letters representing those colors


In [ ]:
# Invoke your newly created function

Now we can _apply_ this function to our `pd.Series` object, returning the result (which we can use to overwrite the source, if we choose).

In [ ]:
# With apply


The `pd.DataFrame.apply` implementation is similar, however it effectively 'scrolls through' the columns and passes each one sequentially to your function:

```python
Objects passed to the function are Series objects whose index is
either the DataFrame index (axis=0) or the DataFrame columns (axis=1).
```

It should only be used when you wish to apply the same function to all columns (or rows) of your `pd.DataFrame` object.

We can also use `pd.Series.apply()` with a **lambda expression**. This is an undeclared function and is commonly used for simple functions within the `.apply()` method. Let's use it to add $100 to our `ListPrice` column. Hey, baby needs new shoes!

In [ ]:
# Without apply - using a lambda

In [ ]:
# And now with 100 more dollars!

Boom! Maybe financing that new boat wasn't such a bad idea after all!

**Your turn:** Identify one other column where we may want to write a new apply function, or use the one we just created for the purposes of cleaning up our dataset.

In [ ]:
# Identify a column to change


In [ ]:
# Write a function to mutate that column (or columns) 


In [ ]:
# Apply that function across the whole column


---
<h1>Joining Table with Pandas</h1>

Pandas provides support for combining `Series`, `DataFrame` and even `xarray` (3 dimensional `DataFrame`s, formerly known in pandas v0.20.0 as `Panel`s) objects with various kinds of set logic for the indicies and relational algebra functionality in the case of join / merge-type operations. More simply stated, this allows you to combine `DataFrame`s.

<!-- Overview -->
<details>
    <summary>Overview</summary>
    <ul>
        <li><b>In this session, we'll cover:</b></li>
        <br>
        <ul>
            <li>Concatenating objects with <code>.append()</code> and <code>.concat()</code></li>
            <li>Combining objects with <code>.join()</code> and <code>.merge()</code></li>
            <li>Combining timeseries objects with <code>.merge_ordered()</code></li>
            <li>Traditionally, this functionality is performed in a relational database, such as <a href="https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html#compare-with-sql-join">SQL</a>. With pandas, you'll be able to perform the same operations - in python! The backend is <code>numpy</code>, a powerful linear algebra library which helps keep things speedy</li>
        </ul>
        <br>
        <li><b>Why Join?</b></li>
        <br>
        <ul>
            <li>You might be asking yourself - why keep data separated in different files? <i>Why not just keep it all in one file?</i></li>
            <li>The answer stems from a thing called <a href="https://support.microsoft.com/en-us/help/283878/description-of-the-database-normalization-basics">database normalization</a>. When a database is <i>normalized</i>, it is structured in such a way that redundancy of data is minimized. This allows a database to be faster, smaller, and more flexible when it comes time to change the data inside of it</li>
            <li>The manifestation of this <i>normalization</i> is data that is represented within multiple <a href="https://en.wikipedia.org/wiki/Table_(database)">tables</a> (which are effectively dataframes), related to each other by <a href="https://www.studytonight.com/dbms/database-key.php">keys</a>, or columns in one table that equal a column in another table, allowing them to be joined. In this case, our tables are the <code>.csv</code> files we'll be importing</li>
        </ul>
    </ul>
</details>

<!-- TOC -->
<details>
    <summary>Table of Contents</summary>
    <ul>
        <li><a href="#conapp">Concatenate and Append</a></li>
        <ul>
            <li><a href="#concatenate">Concatenate</a></li>
            <li><a href="#append">Append</a></li>
        </ul>
        <li><a href="#joining">Joining</a></li>
        <ul>
            <li><a href="join">Join</a></li>
            <li><a href="#merge">Merge</a></li>
            <ul>
                <li><a href="#merge_keycols">Merge on Non-Index Columns</a></li>
                <li><a href="#yourturn">Now it's Your Turn!</a></li>
            </ul>
        </ul>
        <li><a href="#exercise">Exercise - AdventureWorks</a></li>
        <ul>
            <li><a href="#p_exercise">Table Joins on Live Data</a></li>
            <ul>
                <li><a href="#ex_pp">Join Product Tables</a></li>
                <li><a href="#ex_soh_sod">Join Sales Order Header and Sales Order Detail Tables</a></li>
                <li><a href="#ex_soh_sod_pt">Join Sales Order Header, Sales Order Detail, and Product Tables</a></li>
            </ul>
        </ul>
    </ul>
</details>

<div id="conapp"></div>
<h2>Concatenate and Append</h2>

<div id="concatenate"></div>
<h3>Concatenate</h3>

Concatenate sticks dataframes together, either on top of each other, or next to each other.

```python
Signature: pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)
Docstring:
Concatenate pandas objects along a particular axis with optional set logic
along the other axes.
```

First, let's create two dataframes, `df1` and `df2`.

In [ ]:
# KEEP
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df1.head()

In [ ]:
# KEEP
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])
df2.head()

Next, let's stick the dataframes on top of each other using `concat`. 

Finally, let's stick the dataframes <b>next</b> to each other using `concat`. Use of the `axis` kwarg will help us here.

<div id="append"></div>
<h3>Append</h3>

Append is very similar to `concat`, except it limits itself to a specific case of `concat`, where `axis=0` (stack on top of each other) and `join=outer` (how to handle the axis of the second dataframe). For almost all cases, `concat` has all the functionality of `append` (and more) and can replace `append` entirely.

```python
Signature: pd.DataFrame.append(self, other, ignore_index=False, verify_integrity=False, sort=None)
Docstring:
Append rows of `other` to the end of this frame, returning a new
object. Columns not in this frame are added as new columns.
```

Also note that `append` is a DataFrame and Series method, and not a pandas library function like `concat` is.

<div id="joining"></div>
<h2>Joining</h2>

<div id="join"></div>
<h3>Join</h3>

`join` allows us to compare two dataframes, and combine them by using a matching column known as a `key`. Normally, during joins, this key is explicitly stated (we'll get to this with `merge` in our next example). With `join`, the `key` joining the table is always the `index` of the first table with (by default) the index of the second table. 

```python
Signature: pd.DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
Docstring:
Join columns with other DataFrame either on index or on a key
column. Efficiently Join multiple DataFrame objects by index at once by
passing a list.
```

First, let's create two dataframes.

In [ ]:
# KEEP
df1 = pd.DataFrame([['a', 1], ['b', 2], ['c', 3], ['d', 4]], columns=['letter', 'number'])
df1.head()

In [ ]:
# KEEP
df2 = pd.DataFrame([['e', 5], ['f', 6]], columns=['letter', 'number'])
df2.head()

Now, lets `join` these two dataframes. Note that we will `key`, or 'line up', the two dataframes based on their `indicies`.

Note that, when joining dataframes with any common column names, we will need to supply a `lsuffix` or `rsuffix` kwarg. This is appended to the end of the column name of the returned, joined dataframe to differentiate and identify the source column. Here, we'll use `_df1` to identify that the column shown came from the `df1` dataframe, and `_df2` as a suffix to identify its origin as the `df2` dataframe. 

Note how we have joined the two dataframes on their indicies, which creates a null for rows of index 2 and 3 in `df2`. This is expected and correct.

Also note that the default join behavior of `join` is `left`. We can change this with the `how` kwarg.

For reference, here are the common types of joins. Join types won't be covered in this lesson.
<p align="center">
<img width="500px" src="https://i.stack.imgur.com/udQpD.jpg">
</p>

The type of join we performed above is shown in the upper-left most figure in the above chart.

<div id="merge"></div>
<h3>Merge</h3>

Similar to `join` is `merge`. The difference between the two is the <i>keying behavior</i>. `merge` has a richer API (more functionality) and allows one to join on columns in the source dataframe <i>other than the index</i>. Because `merge` can effectively do everything that `join` can do, and more - it is recommended to always use `merge` unless code brevity is the top concern. 

```python
Signature: pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
Docstring:
Merge DataFrame objects by performing a database-style join operation by
columns or indexes.
```

Note that `merge` is <i>both</i> a DataFrame method as well as a pandas function. Below, we'll be using the pandas function, `pd.merge()`.

Note that we've achieved the same exact output as we did with `join`, but it took a little more explicit work. Let's run through the arguments for clarity:

<ul>
    <li><code>df1</code>: this is the first dataframe, and considered to be on the 'left' of <code>df2</code></li>
    <li><code>df2</code>: this is the second dataframe, considered to be on the right of <code>df1</code></li>
    <li><code>how='left'</code>: this states the type of join; see the above SQL join table</li>
    <li><code>left_index=True</code>: this uses the index of <code>df1</code> as the join key for the left table</li>
    <li><code>right_index=True</code>: this uses the index of <code>df2</code> as the join key for the right table</li>
    <li><code>suffixes</code>: this places <code>_df1</code> after column names which came from <code>df1</code></li>
</ul>

<div id="merge_keycols"></div>
<h4>Merge on Non-Index Columns</h4>

This brings us to our next point: merging on columns that are not the index columns. This is very, very common in SQL joins and this technique can be used to join just about any DataFrame.

First, let's create some more realistic data - stocks!

In [ ]:
# KEEP
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 
                          'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], 
                       '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 
                          'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})

Now, let's join the <code>openprice</code> and <code>wkhigh</code> dataframes together.

Note how our `Symbol` column isn't in the same order in each dataframe. This is intentional, and note that the dataframe on the left, `openprice` dictates the order of the dataframe on the right, `wkhigh`. Also note that the shared key between the two dataframes is exempt from having a <code>suffix</code> applied to it. 

<div id="yourturn"></div>
<h4>Now it's your turn!</h4>

<ul>
    <li><code>merge</code> the <code>openprice</code> and <code>stockname</code> dataframes and inspect the result</li>
    <li><code>merge</code> all three dataframes together and inspect the result</li>
</ul>

<div id="exercise"></div>
<h2>Exercise - Adventure Works</h2>
<p align="right">
<img src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">
</p>

<div id="p_exercise"></div>
<h3>Table Joins on Live Data</h3>

Here are the data dictionaries we'll be using for the following exercise:

<ul>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html">Production.Product</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubCategory.html">Production.ProductSubcategory</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html">Sales.SalesOrderHeader</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html">Sales.SalesOrderDetail</a></li>
</ul>

In [ ]:
p = pd.read_csv('Production.Product.csv', sep='\t')
ps = pd.read_csv('Production.ProductSubcategory.csv', sep='\t')
soh = pd.read_csv('Sales.SalesOrderHeader.csv', sep='\t', nrows=1000)
sod = pd.read_csv('Sales.SalesOrderDetail.csv', sep='\t', nrows=1000)

<div id="ex_pp"></div>
<h4>Join Product Tables</h4>

<ul>
    <li>Using the <code>Production.Product.ProductID</code> and <code>Production.ProductSubcategory.ProductID</code> keys, join the <code>Production.Product</code> and <code>Production.ProductSubcategory</code> tables</li>
</ul>

<div id="ex_soh_sod"></div>
<h4>Join Sales Order Header and Sales Order Detail Tables</h4>

<ul>
    <li>Join the <code>Sales.SalesOrderHeader</code> and <code>Sales.SalesOrderDetail</code> tables</li>
    <li>Don't forget to use your data dictionaries!</li>
</ul>

<div id="ex_soh_sod_pt"></div>
<h4>Join Sales Order Header, Sales Order Detail, and Product Tables</h4>

<ul>
    <li>Join the <code>Sales.SalesOrderHeader</code>, <code>Sales.SalesOrderDetail</code>, and <code>Production.Product</code> tables</li>
    <li>Don't forget to use your data dictionaries!</li>
</ul>